# Данные

In [ ]:
!pip install --quiet datasets

In [ ]:
from datasets import Dataset
import json
from collections import Counter
from sklearn.model_selection import train_test_split

# Читаем датасет

In [ ]:
with open('dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Преобразуем классы

In [ ]:
for d in data:
    if d["label"] == "ANSWER":
        d["label"] = "CHAT"
    elif d["label"] == "SUPPORT":
        d["label"] = "QUESTION"

# Некоторая инфа по таргет рейту данных

In [ ]:
def calc_target_rates(data):
  target_rates = {
    "SPAM": 0,
    "QUESTION": 0,
    "FEEDBACK": 0,
    "CHAT": 0,
    "IMPORTANT": 0
  }
  for dct in data:
    target_rates[dct['label']] += 1
  return target_rates
print(calc_target_rates(data))

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
print(calc_target_rates(train_data))

In [ ]:
print(calc_target_rates(test_data))

In [ ]:
import random

dataset_train = Dataset.from_list(train_data)

test_data_arr = [t["text"] for t in test_data]
dataset_test = Dataset.from_list(test_data)

In [ ]:
dataset_train

In [ ]:
dataset_test

# Параметры

In [ ]:
id2label = {
    0: "SPAM",
    1: "QUESTION",
    2: "FEEDBACK",
    3: "CHAT",
    4: "IMPORTANT"
}

label2id = {v: k for k, v in id2label.items()}

# Модель

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from torch import nn
from transformers import AutoModel, AutoModelForSequenceClassification

class CustomClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, num_labels)
        )

    def forward(self, x):
        return self.layers(x)

class CustomModel(AutoModelForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = AutoModel.from_pretrained(config._name_or_path, config=config)
        hidden_size = config.hidden_size

        self.classifier = CustomClassificationHead(hidden_size, config.num_labels)

    def forward(self, **kwargs):
        outputs = self.bert(**kwargs)
        pooled_output = outputs.pooler_output  # [CLS] токен
        logits = self.classifier(pooled_output)
        return {'logits': logits}

In [ ]:
from transformers import AutoTokenizer, AutoConfig

model_name = "blanchefort/rubert-base-cased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

model = CustomModel.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True).to(device)

# Обучение

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    tokenized["label"] = [label2id[label] for label in examples["label"]]
    return tokenized

tokenized_datasets_train = dataset_train.map(
    tokenize_function,
    batched=True,
    remove_columns=["text", "label"]
)

tokenized_datasets_test = dataset_test.map(
    tokenize_function,
    batched=True,
    remove_columns=["text", "label"]
)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Определяем функцию для расчета метрик
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    precision = precision_score(labels, predictions, average='macro', labels=np.unique(labels))
    recall = recall_score(labels, predictions, average='macro', labels=np.unique(labels))
    f1 = f1_score(labels, predictions, average='macro', labels=np.unique(labels))

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    num_train_epochs=50,
    logging_dir="./logs",
    logging_steps=1000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

trainer.train()

eval_results = trainer.evaluate()

print(f"Evaluation Results: {eval_results}")

# Проверка

In [ ]:
def convert_labels_to_vector(data, label2id):
    return [label2id[item['label']] for item in data]

In [ ]:
test_data_arr[:5]

In [ ]:
y_test = convert_labels_to_vector(test_data, label2id)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
label2id

In [ ]:
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
)

y_pred = []
for text in test_data_arr:
    result = classifier(text)
    y_pred.append(label2id[result[0]['label']])
    #print(f"Текст: {text[:30]}... | Класс: {result[0]['label']}")

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
precision_class_SPAM = precision_score(y_test, y_pred, labels=[0], average='micro')
recall_class_SPAM = recall_score(y_test, y_pred, labels=[0], average='micro')
precision_class_SPAM, recall_class_SPAM

In [ ]:
precision_class_QUESTION = precision_score(y_test, y_pred, labels=[1], average='micro')
recall_class_QUESTION = recall_score(y_test, y_pred, labels=[1], average='micro')
precision_class_QUESTION, recall_class_QUESTION

In [ ]:
precision_class_FEEDBACK = precision_score(y_test, y_pred, labels=[2], average='micro')
recall_class_FEEDBACK = recall_score(y_test, y_pred, labels=[2], average='micro')
precision_class_FEEDBACK, recall_class_FEEDBACK

In [ ]:
precision_class_CHAT = precision_score(y_test, y_pred, labels=[3], average='micro')
recall_class_CHAT = recall_score(y_test, y_pred, labels=[3], average='micro')
precision_class_CHAT, recall_class_CHAT

In [ ]:
precision_class_IMPORTANT = precision_score(y_test, y_pred, labels=[4], average='micro')
recall_class_IMPORTANT = recall_score(y_test, y_pred, labels=[4], average='micro')
precision_class_IMPORTANT, recall_class_IMPORTANT

In [ ]:
precision = precision_score(y_test, y_pred, average='macro', labels=[0,1,2,3,4])
recall = recall_score(y_test, y_pred, average='macro', labels=[0,1,2,3,4])
f1 = f1_score(y_test, y_pred, average='macro', labels=[0,1,2,3,4])
precision, recall, f1